<img src=images/gdd-logo.png width=300px align=right> 

# Creating New Columns

Often you will want to create a new column so that you can use it at a later date.

This notebook covers:

* [Creating new columns: avoid common bad practice](#bad-pract)
* [Using `assign()` to create new columns](#assign)
    * [<mark>Exercise: Create new weight columns</mark>](#ex-weight)
* [Shifting a column](#shifting)
    * [<mark>Exercise: Find the differences</mark>](#ex-diff)
* [Other methods](#other-verbs)
    * [Calculating the difference](#diff)
    * [Renaming columns](#rename)

First of all, let's load Pandas and the dataset again:

In [ ]:
import pandas as pd

chickweight = pd.read_csv('data/chickweight.csv').rename(str.lower, axis='columns')

## A note on style...

When reading the chickweight data in, the columns are being renamed straight away. Now you can see multiple functions and methods being executed and chained together.

Chaining methods allows you to do multiple things at once, not just renaming columns but also adding columns, filtering rows, aggregating data. Therefore your coding lines are going to become very long.

For this reason, it is best to add brackets around you code, so that you can use multiple lines. Note that you cannot write Python on multiple lines unless it is in some form of brackets!

In [ ]:
chickweight = (
    pd.read_csv('data/chickweight.csv')
    .rename(str.lower, axis='columns') # add a new line at each new method 
)

<a id='bad-pract'></a>
## Creating new columns: avoid common bad practice

First let's add a piece of code that explains something about the data:

In [ ]:
print("There are", chickweight.shape[1], "rows in the DataFrame")

Now say you want to create a new column where the weight is doubled.

You could use the assignment tool to create a new column as seen below.

In [ ]:
chickweight['weight_doubled'] = chickweight['weight'] * 2

In [ ]:
chickweight.head()

However, adding columns like this is considered bad practice, as you have modified the original dataframe.

**Code should always perform in the same way regardless of where it is in the project**

Let's rerun that same code from the cell above. Does the output change?

In [ ]:
print("There are", chickweight.shape[1], "rows in the DataFrame")

Now reports are stating something different about this data. This is going to cause confusion with regards to which data is being used and how it has been edited along the way, leading to Pandas frustration...

<img src='images/04_Creating_Columns/panda.gif' width='300px' align='left'>

To avoid this, you **do not want to overwrite your data frame.** This is a philosophy that is being adopted more and more, and being built into pandas itself.

***Thought experiment: "Ok so how about I just copy the DataFrame and make the changes I need"***

In [ ]:
chickweight_temp = chickweight.copy()
chickweight_temp['weight_doubled'] = chickweight_temp['weight'] * 2

<details>
    <summary><font color=blue>How could this lead to issues?</font></summary>

In this case you didn't overwrite the dataframe, but you may end up with too many versions of a dataframe, which is not memory efficient & will also become confusing.

</details>

<mark>***So what's the answer?***</mark>

<a id='assign'></a>
## Using `.assign()` to create new columns
You can tell pandas to make a new column with `.assign()`, and specify **how** to to calculate with a lambda function.

In [ ]:
(
    chickweight
    .assign(weight_doubled = chickweight['weight'] * 2)
).head()

Note that the original dataframe is unchanged.

In [ ]:
chickweight.head()

But how can you use this new column? For example - creating a new column called weight_quadrupled. The below would cause an error:

In [ ]:
(
    chickweight
    .assign(weight_doubled = chickweight['weight'] * 2)
    .assign(weight_doubled = chickweight['weight_doubled'] * 2)
).head()

This is where the anonymous function `lambda` can come into play! 

Let's make a lambda function with a `DataFrame` as its argument. Typically "nameless" DataFrames get given the shorthand `df`.

In [ ]:
my_lambda_function = lambda df: df['weight_doubled'] * 2

Calling this lambda function looks like this:

In [ ]:
my_lambda_function(chickweight)

Now this logic can be used inside of the assign method:

In [ ]:
(
    chickweight
    .assign(weight_doubled = chickweight['weight'] * 2)
    .assign(weight_doubled = lambda df: df['weight_doubled'] * 2)
).head()

It can even be added to the same assign method, as long as it is being added after `weight_doubled` is created.

In [ ]:
(
    chickweight
    .assign(weight_doubled = chickweight['weight'] * 2, 
            weight_quadrupled = lambda df: df['weight_doubled'] * 2)
).head()

In fact, for the sake of consistency and scalability, using a lambda function is always recommended:

In [ ]:
(
    chickweight
    .assign(weight_doubled = lambda df: df['weight'] * 2, 
            weight_quadrupled = lambda df: df['weight_doubled'] * 2)
).head()

<a id=ex-weight></a>

### <mark>Exercise: Make new weight columns</mark>

1. Assuming that the chick weights are in grams, can you add a column that gives the chickweights in kg?
2. In the same `.assign()`, also add the chickweights in pounds.

*1000 g = 1 kg = 2.205 pounds*

In [ ]:
# %load answers/04_Creating_Columns/new-column.py

## Dropping columns

Note that you can also drop columns if required!

In [ ]:
(
    chickweight
    .drop(columns = ['rownum', 'time'])
).head()

# Conclusion

The `.assign()` method is best practice when it comes to creating new columns. Dataframes **are mutable objects** so it is important to be careful when creating new columns or making any changes that you don't accidentally change the original dataframe.